In [1]:
import sys
print(sys.executable)

/opt/tljh/user/bin/python


In [2]:
# !{sys.executable} -m pip install pandas

### Loading API-Keys
The next step is to load the credentials in order to access the OpenAI API. We use `dotenv` to read the required environment variables from a `.env` file. As a fallback solution we prompt the user to type it in.

In [3]:
# This will load the environment variables from the .env file. Make sure that you have the .env file in the same directory as this notebook.
# A template .env can be found in the file .env.template
%load_ext dotenv
%dotenv

# Initialize Azure OpenAI Service client

In [4]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(azure_deployment="gpt-4o-mini")
temperature = 0.8

# Initialize Azure OpenAI Embeddings client


In [5]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-large",
)

Adicionando memória

In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver


# Create the agent
memory = InMemorySaver()
agent_executor = create_react_agent(model, tools=[], checkpointer=memory)

Prompt de comando compartilhado entre PDF e CSV

In [19]:

shared_prompt_template = """
Você é um assistente técnico especializado em engenharia. Um usuário perguntou:

"{phrase}"

Abaixo estão trechos de projetos que podem conter a resposta:

{context}

Seu objetivo é:
- Localizar se algum projeto (CS, BR_CS, número ou nome) menciona componentes relacionados ao que foi perguntado.
- Dizer diretamente qual é o número de CS ou o nome do projeto correspondente.
- Se houver mais de um possível projeto, mencione os mais relevantes.
- Caso mais detalhes sejam solicitidados, dê preferência a descrições técnicas dos PDF
- Caso materiais específicos sejam citados, dê preferência a busca no CSV

Não informe de onde a informação veio. Seja direto.
Responda no idioma em que foi perguntado
""".strip()



Conversão do CSV em variável vetorizada

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import CSVLoader
from langchain.schema import Document
import pandas as pd

# === 1. Leitura do CSV agrupado ===
df = pd.read_csv("CSV_italo/agrupado.csv", encoding="utf-8")

# === 2. Criação dos documentos (um por linha) ===
documents = [
    Document(
        page_content=row.to_string(index=False),
        metadata={"Projeto": row["Projeto"] if "Projeto" in row else "Desconhecido"}
    )
    for _, row in df.iterrows()
]

# === 3. Divisão dos documentos em chunks menores ===
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    add_start_index=True,
)

chunks = text_splitter.split_documents(documents)

# === 4. Vetorização e persistência com Chroma ===
vector_store_csv = Chroma(
    embedding_function=embeddings,
    collection_name="agrupado_CSV",
    persist_directory="./CSV_VectorStore"
)

document_ids = vector_store_csv.add_documents(chunks)

print(f"{len(document_ids)} chunks vetorizados com sucesso.")


/tmp/ipykernel_969901/2219314198.py:29: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store_csv = Chroma(


7119 chunks vetorizados com sucesso.


Convertendo PDF em variável vetorizada

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Caminho da pasta onde estão os PDFs
pdf_folder_path = "./PDFs_Italo"

# Lista para armazenar todos os documentos dos PDFs
all_docs = []

# Itera por todos os arquivos na pasta
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_folder_path, filename)
        try:
            loader = PyPDFLoader(file_path)
            docs = loader.load()
            all_docs.extend(docs)
            print(f"✔ Arquivo carregado: {filename} ({len(docs)} páginas)")
        except Exception as e:
            print(f"⚠ Erro ao carregar '{filename}': {e}")

print(f"\n✅ Total de páginas carregadas: {len(all_docs)}")

# Quebra os documentos em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)

all_splits = text_splitter.split_documents(all_docs)

# Criação do vetor usando Chroma
vector_store_pdf = Chroma(
    embedding_function=embeddings,
    collection_name="projetos_PDF",
    persist_directory= "./VectorStore_PDF"
)
document_ids = vector_store_pdf.add_documents(all_splits)
#document_ids = vector_store.add_documents(cut_info)
#print(f"{len(document_ids)} documentos vetorizados.")

results = vector_store_pdf.similarity_search("o que são paineis eletropneumáticos?", k=3)  # Answer can be found on page 13.


Definindo PDF tool

In [10]:
from typing import Optional
from langchain_core.tools import BaseTool
from langchain_core.tools.base import ArgsSchema
from pydantic import BaseModel, Field, Extra
from langchain.schema import Document

class GetPDFInput(BaseModel):
    phrase: str = Field(description="Question to ask about PDF documents")

class GetPDFTool(BaseTool):
    name: str = "get_pdfs"
    description: str = "Search for information in vectorized PDFs"
    args_schema: Optional[ArgsSchema] = GetPDFInput

    class Config:
        extra = Extra.allow

    def __init__(self, vectorstore, model):
        super().__init__()
        self.vectorstore = vectorstore
        self.model = model

    def _run(self, phrase: Optional[str] = None) -> str:
        retrieved_docs = self.vectorstore.similarity_search(phrase, k=3)
        context = "\n".join(doc.page_content for doc in retrieved_docs)

        prompt = shared_prompt_template.format(phrase=phrase, context=context)
        resposta = self.model.invoke(prompt)
        return resposta.content.strip()



/tmp/ipykernel_969901/1627442536.py:16: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.allow


Definindo CSV tool

In [11]:
from typing import Optional
from langchain_core.tools import BaseTool
from langchain_core.tools.base import ArgsSchema
from pydantic import BaseModel, Field, Extra

from langchain.schema import Document

class GetCSVInput(BaseModel):
    phrase: str = Field(description="Question to ask about structured CSV project data")

class GetCSVTool(BaseTool):
    name: str = "get_csv"
    description: str = "Search for information in vectorized CSV project data"
    args_schema: Optional[ArgsSchema] = GetCSVInput

    class Config:
        extra = Extra.allow

    def __init__(self, vectorstore, model):
        super().__init__()
        self.vectorstore = vectorstore
        self.model = model

    def _run(self, phrase: Optional[str] = None) -> str:
        retrieved_docs = self.vectorstore.similarity_search(phrase, k=3)
        context = "\n".join(doc.page_content for doc in retrieved_docs)

        prompt = shared_prompt_template.format(phrase=phrase, context=context)
        resposta = self.model.invoke(prompt)
        return resposta.content.strip()


/tmp/ipykernel_969901/2756060632.py:17: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.allow


Tool para unificar respostas

In [12]:
from typing import Optional
from langchain_core.tools import BaseTool
from langchain_core.tools.base import ArgsSchema
from pydantic import BaseModel, Field, Extra

class UnifiedInput(BaseModel):
    phrase: str = Field(description="Question to ask about PDF and CSV documents")

class UnifiedSearchTool(BaseTool):
    name: str = "unified_project_search"
    description: str = "Unified search for project data in both PDFs and CSVs"
    args_schema: Optional[ArgsSchema] = UnifiedInput

    class Config:
        extra = Extra.allow

    def __init__(self, vectorstore_pdf, vectorstore_csv, model):
        super().__init__()
        self.vectorstore_pdf = vectorstore_pdf
        self.vectorstore_csv = vectorstore_csv
        self.model = model

    def _run(self, phrase: Optional[str] = None) -> str:
        # Buscar documentos similares nos dois vetores
        docs_pdf = self.vectorstore_pdf.similarity_search(phrase, k=15)
        docs_csv = self.vectorstore_csv.similarity_search(phrase, k=15)

        # Junta os textos dos documentos
        combined_context = "\n".join(doc.page_content for doc in (docs_pdf + docs_csv))

        # Prepara o prompt
        prompt = shared_prompt_template.format(phrase=phrase, context=combined_context)

        # Gera resposta
        resposta = self.model.invoke(prompt)
        return resposta.content.strip()


/tmp/ipykernel_969901/4007489383.py:15: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.allow


Executa agente

In [21]:
from langchain_core.messages import HumanMessage
import time

# Inicializa a ferramenta unificada
unified_tool = UnifiedSearchTool(
    vectorstore_pdf=vector_store_pdf,
    vectorstore_csv=vector_store_csv,
    model=model
)

# Loop de execução
while True:
    user_input = input("Você: ")
    if user_input.lower() == "exit":
        break

    message = HumanMessage(content=user_input)
    message.pretty_print()
    config = {"configurable": {"thread_id": "abc123"}}
    result = unified_tool.invoke({"phrase": user_input}, config)
    print("\n📄 Resultado:")
    print(result)
    time.sleep(1)


================================ Human Message =================================

pode me encontrar algum projeto com caixa em aço carbono, com dimensões próximas a 800x800x300?

📄 Resultado:
O projeto que contém uma caixa em aço carbono com dimensões próximas a 800x800x300mm é o **BR_CS.2214916**, que especifica uma caixa de montagem com dimensões estimadas de **760x760x300mm**. 

Caso precise de mais detalhes técnicos, posso fornecer a descrição técnica deste projeto.
================================ Human Message =================================

pode me descrever o que há nesse projeto?

📄 Resultado:
O projeto mencionado possui informações relacionadas ao escopo de fornecimento, cronograma de execução e aprovações, assim como aspectos da documentação. Para mais detalhes, recomenda-se consultar as descrições técnicas dos PDFs associados ao projeto. Se precisar de mais informações específicas ou detalhes técnicos, avise-me!
================================ Human Message ============